# Experiment 087: Stacking with Meta-Learner

**Strategy**: Instead of simple weighted averaging, use proper stacking:
1. Train diverse base models (GP, MLP, LGBM, CatBoost, XGBoost)
2. Use out-of-fold predictions as features for a meta-learner
3. Meta-learner can be Ridge or simple linear model

**Hypothesis**: Stacking might change the CV-LB relationship by learning how to combine models optimally for different input regions.

**Key insight**: All previous approaches fall on the same CV-LB line (LB = 4.31*CV + 0.0525). We need to CHANGE this relationship, not just improve CV.

In [1]:
# Standard imports
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern, WhiteKernel, ConstantKernel
import lightgbm as lgb
import catboost as cb
import xgboost as xgb
import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)
torch.manual_seed(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.set_default_dtype(torch.double)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cuda


In [2]:
# Data loading functions
DATA_PATH = '/home/data'

INPUT_LABELS_NUMERIC = ["Residence Time", "Temperature"]
INPUT_LABELS_SINGLE_SOLVENT = ["Residence Time", "Temperature", "SOLVENT NAME"]
INPUT_LABELS_FULL_SOLVENT = ["Residence Time", "Temperature", "SOLVENT A NAME", "SOLVENT B NAME", "SolventB%"]

def load_data(name="full"):
    if name == "full":
        df = pd.read_csv(f'{DATA_PATH}/catechol_full_data_yields.csv')
        X = df[INPUT_LABELS_FULL_SOLVENT]
    else:
        df = pd.read_csv(f'{DATA_PATH}/catechol_single_solvent_yields.csv')
        X = df[INPUT_LABELS_SINGLE_SOLVENT]
    Y = df[["Product 2", "Product 3", "SM"]]
    return X, Y

def generate_leave_one_out_splits(X, Y):
    for solvent in sorted(X["SOLVENT NAME"].unique()):
        mask = X["SOLVENT NAME"] != solvent
        yield (X[mask], Y[mask]), (X[~mask], Y[~mask])

def generate_leave_one_ramp_out_splits(X, Y):
    ramps = X[["SOLVENT A NAME", "SOLVENT B NAME"]].drop_duplicates()
    for _, row in ramps.iterrows():
        mask = ~((X["SOLVENT A NAME"] == row["SOLVENT A NAME"]) & (X["SOLVENT B NAME"] == row["SOLVENT B NAME"]))
        yield (X[mask], Y[mask]), (X[~mask], Y[~mask])

print('Data loading functions defined')

Data loading functions defined


In [3]:
# Load feature lookups
SPANGE_DF = pd.read_csv(f'{DATA_PATH}/spange_descriptors_lookup.csv', index_col=0)
DRFP_DF = pd.read_csv(f'{DATA_PATH}/drfps_catechol_lookup.csv', index_col=0)
ACS_PCA_DF = pd.read_csv(f'{DATA_PATH}/acs_pca_descriptors_lookup.csv', index_col=0)

# Filter DRFP to high-variance columns
drfp_variance = DRFP_DF.var()
nonzero_variance_cols = drfp_variance[drfp_variance > 0].index.tolist()
DRFP_FILTERED = DRFP_DF[nonzero_variance_cols]

print(f'Spange: {SPANGE_DF.shape}, DRFP filtered: {DRFP_FILTERED.shape}, ACS PCA: {ACS_PCA_DF.shape}')

Spange: (26, 13), DRFP filtered: (24, 122), ACS PCA: (24, 5)


In [4]:
# Full Featurizer
class FullFeaturizer:
    def __init__(self, mixed=False):
        self.mixed = mixed
        self.spange_df = SPANGE_DF
        self.drfp_df = DRFP_FILTERED
        self.acs_pca_df = ACS_PCA_DF
        self.feats_dim = 2 + 3 + self.spange_df.shape[1] + self.drfp_df.shape[1] + self.acs_pca_df.shape[1]

    def featurize(self, X, flip=False):
        X_vals = X[INPUT_LABELS_NUMERIC].values.astype(np.float64)
        temp_c = X_vals[:, 1:2]
        time_m = X_vals[:, 0:1]
        temp_k = temp_c + 273.15
        inv_temp = 1000.0 / temp_k
        log_time = np.log(time_m + 1e-6)
        interaction = inv_temp * log_time
        X_kinetic = np.hstack([X_vals, inv_temp, log_time, interaction])
        
        if self.mixed:
            A_spange = self.spange_df.loc[X["SOLVENT A NAME"]].values
            B_spange = self.spange_df.loc[X["SOLVENT B NAME"]].values
            A_drfp = self.drfp_df.loc[X["SOLVENT A NAME"]].values
            B_drfp = self.drfp_df.loc[X["SOLVENT B NAME"]].values
            A_acs = self.acs_pca_df.loc[X["SOLVENT A NAME"]].values
            B_acs = self.acs_pca_df.loc[X["SOLVENT B NAME"]].values
            pct = X["SolventB%"].values.reshape(-1, 1)
            X_spange = A_spange * (1 - pct) + B_spange * pct
            X_drfp = A_drfp * (1 - pct) + B_drfp * pct
            X_acs = A_acs * (1 - pct) + B_acs * pct
        else:
            X_spange = self.spange_df.loc[X["SOLVENT NAME"]].values
            X_drfp = self.drfp_df.loc[X["SOLVENT NAME"]].values
            X_acs = self.acs_pca_df.loc[X["SOLVENT NAME"]].values
        
        return np.hstack([X_kinetic, X_spange, X_drfp, X_acs])

print(f'Full feature dimension: {FullFeaturizer().feats_dim}')

Full feature dimension: 145


In [5]:
# Stacking Ensemble with Meta-Learner
class StackingEnsemble:
    def __init__(self, data='single'):
        self.data = data
        self.mixed = (data == 'full')
        self.featurizer = FullFeaturizer(mixed=self.mixed)
        self.scalers = [StandardScaler() for _ in range(3)]  # For targets
        self.feature_scaler = StandardScaler()
        
        # Base models (will be trained per target)
        self.base_models = {}  # {target_idx: [model1, model2, ...]}
        self.meta_models = []  # One per target
        
    def _create_base_models(self):
        """Create fresh base models for each target."""
        models = []
        
        # 1. CatBoost
        models.append(('catboost', cb.CatBoostRegressor(
            iterations=300,
            learning_rate=0.05,
            depth=6,
            l2_leaf_reg=3,
            random_seed=42,
            verbose=False
        )))
        
        # 2. XGBoost
        models.append(('xgboost', xgb.XGBRegressor(
            n_estimators=300,
            learning_rate=0.05,
            max_depth=6,
            reg_lambda=3,
            random_state=42,
            verbosity=0
        )))
        
        # 3. LightGBM
        models.append(('lgbm', lgb.LGBMRegressor(
            n_estimators=300,
            learning_rate=0.05,
            max_depth=6,
            reg_lambda=3,
            random_state=42,
            verbose=-1
        )))
        
        # 4. Ridge (simple linear model)
        models.append(('ridge', Ridge(alpha=1.0)))
        
        return models
    
    def train_model(self, X, Y):
        """Train stacking ensemble with out-of-fold predictions."""
        X_feats = self.featurizer.featurize(X)
        X_feats = self.feature_scaler.fit_transform(X_feats)
        Y_vals = Y.values
        n_samples = len(Y_vals)
        
        # Scale targets
        Y_scaled = np.zeros_like(Y_vals)
        for i in range(3):
            Y_scaled[:, i] = self.scalers[i].fit_transform(Y_vals[:, i:i+1]).ravel()
        
        # For each target, train base models and collect OOF predictions
        self.base_models = {}
        self.meta_models = []
        
        for target_idx in range(3):
            y = Y_scaled[:, target_idx]
            
            # Create base models
            base_models = self._create_base_models()
            n_models = len(base_models)
            
            # OOF predictions for meta-learner
            oof_preds = np.zeros((n_samples, n_models))
            
            # 5-fold CV for OOF predictions
            from sklearn.model_selection import KFold
            kf = KFold(n_splits=5, shuffle=True, random_state=42)
            
            for fold_idx, (train_idx, val_idx) in enumerate(kf.split(X_feats)):
                X_train_fold = X_feats[train_idx]
                y_train_fold = y[train_idx]
                X_val_fold = X_feats[val_idx]
                
                for model_idx, (name, model) in enumerate(base_models):
                    # Clone model for this fold
                    if name == 'catboost':
                        fold_model = cb.CatBoostRegressor(
                            iterations=300, learning_rate=0.05, depth=6,
                            l2_leaf_reg=3, random_seed=42+fold_idx, verbose=False
                        )
                    elif name == 'xgboost':
                        fold_model = xgb.XGBRegressor(
                            n_estimators=300, learning_rate=0.05, max_depth=6,
                            reg_lambda=3, random_state=42+fold_idx, verbosity=0
                        )
                    elif name == 'lgbm':
                        fold_model = lgb.LGBMRegressor(
                            n_estimators=300, learning_rate=0.05, max_depth=6,
                            reg_lambda=3, random_state=42+fold_idx, verbose=-1
                        )
                    else:  # ridge
                        fold_model = Ridge(alpha=1.0)
                    
                    fold_model.fit(X_train_fold, y_train_fold)
                    oof_preds[val_idx, model_idx] = fold_model.predict(X_val_fold)
            
            # Train final base models on all data
            trained_base_models = []
            for name, model in base_models:
                model.fit(X_feats, y)
                trained_base_models.append((name, model))
            
            self.base_models[target_idx] = trained_base_models
            
            # Train meta-learner on OOF predictions
            meta_model = Ridge(alpha=0.1)
            meta_model.fit(oof_preds, y)
            self.meta_models.append(meta_model)
    
    def predict(self, X):
        """Make predictions using stacking."""
        X_feats = self.featurizer.featurize(X)
        X_feats = self.feature_scaler.transform(X_feats)
        n_samples = len(X_feats)
        
        preds = []
        for target_idx in range(3):
            # Get predictions from all base models
            base_preds = np.zeros((n_samples, len(self.base_models[target_idx])))
            for model_idx, (name, model) in enumerate(self.base_models[target_idx]):
                base_preds[:, model_idx] = model.predict(X_feats)
            
            # Meta-learner prediction
            pred_scaled = self.meta_models[target_idx].predict(base_preds)
            pred = self.scalers[target_idx].inverse_transform(pred_scaled.reshape(-1, 1)).ravel()
            preds.append(pred)
        
        return torch.tensor(np.stack(preds, axis=1))

print('StackingEnsemble defined')

StackingEnsemble defined


In [ ]:
# Cross-validation on single solvent data
import tqdm

X, Y = load_data("single_solvent")
print(f'Single solvent data: {len(X)} samples, {len(X["SOLVENT NAME"].unique())} solvents')

all_mse = []
for fold_idx, split in tqdm.tqdm(enumerate(generate_leave_one_out_splits(X, Y))):
    (train_X, train_Y), (test_X, test_Y) = split
    
    model = StackingEnsemble(data='single')
    model.train_model(train_X, train_Y)
    
    preds = model.predict(test_X).numpy()
    targets = test_Y.values
    
    mse = np.mean((preds - targets) ** 2)
    all_mse.append(mse)

single_mse = np.mean(all_mse)
print(f'\nSingle Solvent MSE: {single_mse:.6f}')

In [ ]:
# Cross-validation on full data
X_full, Y_full = load_data("full")
print(f'Full data: {len(X_full)} samples')

all_mse_full = []
for fold_idx, split in tqdm.tqdm(enumerate(generate_leave_one_ramp_out_splits(X_full, Y_full))):
    (train_X, train_Y), (test_X, test_Y) = split
    
    model = StackingEnsemble(data='full')
    model.train_model(train_X, train_Y)
    
    preds = model.predict(test_X).numpy()
    targets = test_Y.values
    
    mse = np.mean((preds - targets) ** 2)
    all_mse_full.append(mse)

full_mse = np.mean(all_mse_full)
print(f'\nFull Data MSE: {full_mse:.6f}')

In [ ]:
# Calculate overall CV score
n_single = 656
n_full = 1227
overall_mse = (single_mse * n_single + full_mse * n_full) / (n_single + n_full)

print(f'\n=== Stacking Ensemble Results ===')
print(f'Single Solvent MSE: {single_mse:.6f}')
print(f'Full Data MSE: {full_mse:.6f}')
print(f'Overall MSE: {overall_mse:.6f}')
print(f'\nBest baseline (GP+MLP+LGBM): 0.008298')
print(f'Difference: {(overall_mse - 0.008298) / 0.008298 * 100:.2f}%')

In [ ]:
########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE THIRD LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

import tqdm

X, Y = load_data("single_solvent")

split_generator = generate_leave_one_out_splits(X, Y)
all_predictions = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator)):
    (train_X, train_Y), (test_X, test_Y) = split

    model = StackingEnsemble(data='single')  # CHANGE THIS LINE ONLY
    model.train_model(train_X, train_Y)

    predictions = model.predict(test_X)  # Shape: [N, 3]

    # Move to CPU and convert to numpy
    predictions_np = predictions.detach().cpu().numpy()

    # Add metadata and flatten to long format
    for row_idx, row in enumerate(predictions_np):
        all_predictions.append({
            "task": 0,
            "fold": fold_idx,
            "row": row_idx,
            "target_1": row[0],
            "target_2": row[1],
            "target_3": row[2]
        })

# Save final submission
submission_single_solvent = pd.DataFrame(all_predictions)

########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE THIRD LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

In [ ]:
########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE SECOND LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

X, Y = load_data("full")

split_generator = generate_leave_one_ramp_out_splits(X, Y)
all_predictions = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator)):
    (train_X, train_Y), (test_X, test_Y) = split

    model = StackingEnsemble(data='full')  # CHANGE THIS LINE ONLY
    model.train_model(train_X, train_Y)

    predictions = model.predict(test_X)  # Shape: [N, 3]

    # Move to CPU and convert to numpy
    predictions_np = predictions.detach().cpu().numpy()

    # Add metadata and flatten to long format
    for row_idx, row in enumerate(predictions_np):
        all_predictions.append({
            "task": 1,
            "fold": fold_idx,
            "row": row_idx,
            "target_1": row[0],
            "target_2": row[1],
            "target_3": row[2]
        })

# Save final submission
submission_full_data = pd.DataFrame(all_predictions)

########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE SECOND LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

In [ ]:
########### DO NOT CHANGE ANYTHING IN THIS CELL #################
########### THIS MUST BE THE FINAL CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

submission = pd.concat([submission_single_solvent, submission_full_data])
submission = submission.reset_index()
submission.index.name = "id"
submission.to_csv("/home/submission/submission.csv", index=True)

########### DO NOT CHANGE ANYTHING IN THIS CELL #################
########### THIS MUST BE THE FINAL CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################